#Introduction 

This notebook will show how to run a batch of lightcurves (e.g. a campaign) through our entire detection program.

#Find Eclipsing Binaries 

The first task is identify eclipsing binaries so that we can hopefully discover a planet around them. To do this we have a few steps:
<ol>
<li> Run box-least squares on all the lightcurves to determine what periodic signals are in data </li>
<li> Create a subset of EB candidates based on some selection criteria (e.g. depth, period, and strength of periodic signal) </li>
<li> Individually confirm each EB noting fundamental paramaters: period, phase width of the primary and secondary eclipses, the separation fo the primary and secondary eclipses, and the central time of an eclipse
</ol>

##Running box-least squares 

I downloaded <a href = https://github.com/dfm/python-bls> python bindings </a> (Ruth Angus and Daniel Foreman-Mackey) for BLS written originally by Kovacs et al. in Fortran. I have written just a simple wrapper for it (run_bls.py) that allows us to use default parameters (set manually in the code) to look at a large set of lightcurves with multi-core methods. 

How to run:
    1. generate a list of lightcurves with absolute paths in a text file
    2. Make sure retrieve in data.py is suited for your use (i.e. it is
    lightcurves from Foreman-Mackey formatting, not straight from STSCI)
    3. Determine an initial_time that you want to include all lightcurve data
    after from (e.g. in Campaign 2 the first several data points before 2456900
    were always bad. If you give that time in JD anything before then is not
    included in the BLS analysis
    4. Decide where to save the files. 
    5. nbins is the number of windows in the BLS code, used 5 for now. 
    6. Run from the command line:
        python run_bls.py /path/to/your/list/of/lcs #ofCAMPAIGN INITIAL_TIME
        /where/to/save/the/output nbin

        ex:
        python run_bls.py /k2_data/all_c2_lcs 2 2456900 /k2_data/c2_eb/bls.pkl
        5

Note the output is a pickled list of tuples. Where each tuple is the
abbreviated BLS response for that EPIC: (EPIC, (best_period, best_power, depth,
q, in1, in2)) where 
best_period is the best-fit period in the same units as time,
best_power is the power at best_period,
depth is the depth of the transit at best_period,
q is the fractional transit duration,
in1 is the bin index at the start of transit, and
in2 is the bin index at the end of transit.


##Filter EBs 

Since each campaign has thousands of lightcurves, we can't individually determine if they're all EBs. You can do something like the following to generate a list of EB candidates from the BLs results. 

In [ ]:
import sys, pickle, os
import numpy as np
path = '/k2_data/'
sys.path.append(path)
import swarced as sw
from astropy.io import ascii
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [2]:
bls= pickle.load(open("/path/to/output/from/bls/above",'r'))
#Example: c1_eb = pickle.load(open("/Volumes/k2_data/c1_eb/bls.pkl",'r'))
#The above list of tuples is now (epic, (best_period, best_power, depth, q, in1, in2)) where:
#best_period is the best-fit period in the same units as time,best_power is the power at best_period,
#depth is the depth of the transit at best_period,q is the fractional transit duration,
#in1 is the bin index at the start of transit, and in2 is the bin index at the end of transit.
def is_Valid(result):
        #True if Depth is positive and period is less than 30
        return all([(result[1][3] > 0),(result[1][1]<30)])
ebcandidates = np.array([result for result in bls if is_Valid(result)])
strengths = [r[1][2] for r in ebcandidates]
ebcandidates = ebcandidates[strengths.argsort()] #sort by strength of detection
pickle.dump(ebcandidates[-600:], open("file_path", 'w'))

For a more interactive way including plotting and picking a strength cutoff instead of just picking a number of features see the notebook pick_eb_c1 or pick_eb_c3

##Confirm EBs 

There are hopefully many false positives in your list so that you can assure you don't have any false negatives. That means we need to go through as humans and look at each lightcurve. I've written a little tool to take care of this too. It's interactive_clip_limits.py

How to run: 
    1. BLS must be previously. Ideally you have filtered the results according
    to stepwise_execution_template.ipynb. The output of filtering is at
    blsreportpath. 
    2. limitsreportpath is where you want to save the results of your
    interactive clipping
    3. Directory is wherever /k2_data/ has been mounted (e.g.
    /Volumes/k2_data/, /k2_data/, /mnt/k2_data/, ~/k2/)
    4. Run this by:
        python interactive_clip_limits.py /path/to/bls /path/to/limits
        /k2_data/
        ex: python interactive_clip_limits.py /k2_data/c3_eb/ebcandidates.pkl
        /k2_data/c3_eb/interactive_results.pkl /k2_data/

Features:
    The output of this is a pickled array of lists structured:
    [EPIC ID, phase_width_of_primary, phase_width_of_secondary, period,
    separation_between_primary_and_secondary, central_time_of_primary, mode]
    Mode is which radio button you've selected for that file. 
    The other nice feature of this is that you can exit while doing your
    clipping and take a break. When you return (assuming you give it the exact
    same commmand to run) your program will pick back up where you were. 



## Generate a list of final EBs 

With all of this done, we want to make a final list of EBs as an ascii file (e.g. /k2_data/c1_eb/final_eb_list.txt). Load your interactive results.

In [ ]:
interactive = pickle.load(open("/path/to/interactive/results.pkl",'r'))

In [ ]:
interactive = [r for r in interactive if r[-1] == 'EB' or r[-1] == 'EB but check']

In [ ]:
#RETURNNNNNNNNNNNNNNNNNNN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#Mask the EB

We have two methods of masking: simple masking and moving median masking. I'll illustrate moving median masking because that's more what we've been doing.

In [3]:
import multiprocessing as mp

In [4]:
clip_params = ascii.read("/k2_data/c2_eb/final_eb_list.txt")
ls = os.listdir("/k2_data/c2_injection_trial_five/lcs/")
ls = np.array([l for l in ls if "_injected.fits" in l])

<b>ls</b> could also be a list of files read in. You just need filepaths. First we must generate the arguments we're going to clip by. I'm going to use multiprocessing because with a sizeable number of lightcurves this could be slower. 

In [7]:
args = []
for fn in ls:
    if int(fn[4:13]) in clip_params['EPIC']:
        windowsize = 5
        outpath = '/k2_data/c2_injection_trial_five/median_clip/'
        inpath = '/k2_data/c2_injection_trial_five/lcs/'
        epicID, period, t0, pwidth, swidth, sep = clip_params[np.where(int(fn[4:13])==clip_params['EPIC'])[0][0]]
        #epicID, pwidth, swidth, period, sep, t0, mode = int(epicID), float(pwidth), float(swidth), float(period), float(sep), float(t0), str(mode)
        args.append((epicID, 2, period, t0, outpath, sw.C2INITIALTIME, windowsize, '/k2_data/',inpath+ fn, True))

In [8]:
#This function just calls my clip_by_median functino I've already written
def single_clip(args):
    sw.data.clip_by_median(*args)

And now we run it!

In [3]:
start = time.time()
pool = mp.Pool(processes=mp.cpu_count())
results = pool.map(single_clip, args)
pool.close()
pool.join()
end = time.time()

#Build Queries 

The ketu code by Foreman-Mackey needs a query file to give the parameters. In this step, we're also doing some checks to make sure that our lightcurve is even reasonable. Once again, this is for the more complicated injected case. You can get rid of a lot of checks if you're not worried about injections.

In [2]:
path = "/k2_data/c2_injection_trial_five/median_clip/" #where the clipped lightcurves are
q_list = [fn for fn in os.listdir(path) if ".fits" in fn]
ref = ascii.read("/k2_data/c2_eb/final_eb_list.txt") #the final list of EBs with their parameters

In [10]:
for lc in q_list:
    try:
        epic = int(lc[4:13])
        if epic in ref['EPIC']:
            loc = np.where(ref['EPIC']==epic)[0][0]
            if fits.open(path + lc)[3].header['period'] < 70: #the injected planet has a period less than 70, delete for run w/o injections
                ebperiod = ref['period'][loc]
                if ebperiod * np.sqrt(8) < 70: #A planet is possible
                    query = sw.query.get_planet_default("/mnt" + path + lc,"2","/mnt/k2_data/", ebperiod)
                    if query != False:
                        sw.query.save(query, path + lc.split(".fits")[0] + ".query")
    except:
        print lc

ktwo204822463-c02_lpd-lc_040_injected_clipmed.fits


#Run ketu on clipped EBs 

Now we've finally gotten to the point where we can run ketu on the lightcurves! Yay! We have two main ways to do this: 
<ol>
<li><b>multicore on a single machine</b> use multirun_ketu.py and see the top it for description</li>
<li><b>multicore on multiple machines</b> use scoop_ketu.py and see the top of it for run description</li>
</ol>

#Recover ketu results 

Because Marcus is silly, he hasn't yet tested and implemented the instantaneous saving method. (He got confused by some file io problems where it wasn't writing in a synchonous fashion. He understands now. Bug him to fix this!) So, we have to format our own table of results afterwards :(. Just run the below code making sure to change the paths accordingly. He's terribly sorry. 

In [24]:
path = "/k2_data/c2_injection_trial_five/median_clip/"
contents = os.listdir(path)
contents = np.array([fn for fn in contents if ("_injected_clipmed.result" in fn)])
epicid = np.array([fn.split("-")[0][4:] for fn in contents],dtype=np.int32)

In [25]:
fails = []
inj_center, inj_period, inj_rpbyrs, inj_tdepth, inj_tctime, inj_prad, inj_srad, inj_smass, inj_impact = [],[], [], [], [], [], [], [], []
inj_ttimes=[]
lc_remaining = []
fnlist = []
for fn in contents:
    if fn not in fails:
        hdulist = fits.open(path + fn.split(".")[0] + ".fits")
        head = hdulist[3].header
        inj_period += [head['PERIOD']]
        inj_rpbyrs += [head['RRATIO']]
        inj_tdepth += [head['TDEPTH']]
        inj_tctime += [head['TCTIME']+hdulist[1].header['BJDREFI']]
        inj_prad += [head['PRADRJ']]
        inj_srad += [head['SRADRS']]
        inj_smass += [head['MSTAR']]
        inj_impact += [head['IMPACT']]
        inj_center += [hdulist[3].data['center'][0] +hdulist[1].header['BJDREFI']]
        fnlist.append(fn)
        hdulist.close()

In [29]:
epics = []
for fn in contents:
    if fn not in fails:
        epics += [int(fn[4:13])]
epics = np.array(epics)

In [32]:
inj_period, inj_rpbyrs, inj_tdepth=np.array(inj_period), np.array(inj_rpbyrs), np.array(inj_tdepth)                                                   
inj_tctime, inj_prad, inj_srad = np.array(inj_tctime), np.array(inj_prad), np.array(inj_srad)
inj_smass, inj_impact =np.array(inj_smass), np.array(inj_impact)
inj_center = np.array(inj_center)
contents, epicid = np.array(contents), np.array(epicid)
inj_ttimes = np.array(inj_ttimes)
lc_remaining= np.array(lc_remaining)

In [33]:
recovered_period = []
recovered_s2n = []
second_period = []
numpeaks = []
recovered_depthivar, recovered_t0, recovered_phicnorm, recovered_rms, recovered_depth, recovered_depths2n = [],[],[],[],[],[]
for fn in contents:
    with open(path + fn,'r')as f:
        if fn not in fails:
            recovered_period_s, recovered_t0_s, recovered_depth_s, recovered_depths2n_s = [], [], [], []
            result = pickle.load(f)
            for i in range(25):
                if i < len(result['peaks']):
                    recovered_period_s += [result['peaks'][i]['period']]
                    recovered_t0_s += [result['peaks'][i]['t0']]
                    recovered_depth_s += [result['peaks'][i]['depth']]
                    recovered_depths2n_s += [result['peaks'][i]['depth_s2n']]
                else:
                    recovered_period_s += [-1]
                    recovered_t0_s += [-1]
                    recovered_depth_s += [-1]
                    recovered_depths2n_s += [-1]
            numpeaks += [len(result['peaks'])]
            recovered_period.append(recovered_period_s)
            recovered_t0.append(recovered_t0_s)
            recovered_depth.append(recovered_depth_s)
            recovered_depths2n.append(recovered_depths2n_s)
recovered_period = np.array(recovered_period)
numpeaks = np.array(numpeaks)

In [40]:
success = []

In [41]:
for i in range(len(recovered_period)):
    ip = inj_period[i]
    s = False
    for j in range(len(recovered_period[i])):
        if abs(inj_period[i] - recovered_period[i][j]) < 0.1:
            s = True
    success.append(s)

In [42]:
ref = ascii.read("/k2_data/c2_eb/final_eb_list.txt")
eb_period = [ref['period'][np.argmax(ref['EPIC']==epic)] for epic in epics]

In [46]:
f = open("/k2_data/c2_injection_trial_five/med_results.txt",'w') #save somewhere
f.write("EPIC filename success eb_period inj_period inj_period_by_eb_period inj_radius_ratio inj_transit_depth inj_planet_radius inj_star_radius inj_star_mass inj_impact_param inj_center\n")
for i in range(len(epics)):
    #print i
    l = "{0:d} {1:s} {2:d} {3:f} {4:f} {12:f} {5:f} {6:f} {7:f} {8:f} {9:f} {10:f} {11:f}\n".format(epics[i], fnlist[i], success[i], 
                                                                                       eb_period[i], inj_period[i], inj_rpbyrs[i], inj_tdepth[i],
                                                                                      inj_prad[i], inj_srad[i], inj_smass[i], inj_impact[i], 
                                                                                                    inj_center[i], inj_period[i]/eb_period[i])
    
    f.write(l)
f.close()

#Discovering the Planets! 

In cases where you are looking at things other than injected planets, you obviously have to individually examine the lightcurves for planet-like signatures. There is a notebook in nbs called validate_planets_c1.ipynb that shows an example. (Any of the validate notebooks will do the trick!)